In [1]:
# Python script to do the analysis based on the account table
import openpyxl
import numpy as np
import pandas as pd
import os
from util import rs_conn, mail
import time
import datetime
import logging

In [2]:
# Get the redshift user name and password
rs_user = os.getenv('RS_USER')
rs_pass = os.getenv('RS_PASS')

# Connect to the Redshift
rs = rs_conn.RS_CONN(rs_user=rs_user, rs_pass=rs_pass)
rs.set_Conn()

In [3]:
acct_engagement_pull_all_cycle="""
select * from da_stage.vintage_sumry
;
"""
all_df_1=rs.get_result_df(sql_query=acct_engagement_pull_all_cycle)

In [4]:
all_df_1['attrition_pct'] = all_df_1['attrition_cnt']/all_df_1['actived_acct_cnt']
all_df_1['co_pct'] = all_df_1['co_cnt']/ all_df_1['actived_acct_cnt']
all_df_1['retention_pct'] = all_df_1['open_acct_cnt']/ all_df_1['actived_acct_cnt']
all_df_1['revolving_pct'] = all_df_1['tot_adb_amt']/ all_df_1['all_adb_amt']

all_df_1['active_rate_2'] = all_df_1['active_acct_cnt']/all_df_1['actived_acct_cnt']

In [5]:
all_df_1.columns

Index(['run_dt', 'stmt_mth', 'cycle_end_dt', 'vintage', 'stmt_num',
       'actived_acct_cnt', 'attrition_cnt', 'co_cnt', 'other_close_cnt',
       'open_acct_cnt', 'active_acct_cnt', 'active_rate',
       'actived_credit_limit', 'open_credit_limit', 'avg_open_acct_apr',
       'active_credit_limit', 'avg_active_credit_line', 'transactor_acct_cnt',
       'transactor_acct_pct', 'revolver_acct_cnt', 'revolver_acct_pct',
       'transactor_revolver_acct_cnt', 'transactor_revolver_acct_pct',
       'tot_end_os_bal_amt', 'acct_end_os_bal_amt', 'nondq90_end_os_bal_amt',
       'dq90_end_os_bal_amt', 'tot_txn_amt', 'acct_txn_amt', 'txn_cl_util_pct',
       'tot_txn_cnt', 'acct_txn_cnt', 'txn_size_amt', 'tot_adb_amt',
       'acct_adb_amt', 'adb_cl_util_pct', 'interest_bearing_adb_amt',
       'all_adb', 'nondq90_interest_bearing_adb_amt', 'nondq_90all_adb',
       'revolver_bal_pct', 'tot_prncp_amt', 'incremental_open_fee_acct_cnt',
       'nondq90_prncp_amt', 'dq90_prncp_amt', 'co_bal', 'st

In [6]:
actived_acct_cnt = pd.pivot_table(all_df_1,values='actived_acct_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
actived_acct_cnt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,110.0,110.0,111.0,111.0,111.0,111.0,111.0,111.0,112.0,112.0,...,113.0,113.0,113.0,113.0,113.0,113.0,113.0,113.0,113.0,113.0
202002,NaN,1742.0,1742.0,1742.0,1742.0,1742.0,1742.0,1742.0,1742.0,1742.0,...,1742.0,1742.0,1742.0,1742.0,1742.0,1742.0,1742.0,1742.0,1742.0,1742.0
202003,NaN,NaN,6531.0,6531.0,6532.0,6532.0,6532.0,6532.0,6532.0,6532.0,...,6532.0,6532.0,6532.0,6532.0,6532.0,6532.0,6532.0,6532.0,6532.0,6532.0
202004,NaN,NaN,NaN,5310.0,5310.0,5310.0,5311.0,5311.0,5311.0,5311.0,...,5311.0,5311.0,5311.0,5311.0,5311.0,5311.0,5311.0,5311.0,5311.0,5311.0
202005,NaN,NaN,NaN,NaN,2318.0,2318.0,2318.0,2318.0,2318.0,2318.0,...,2318.0,2318.0,2318.0,2318.0,2318.0,2318.0,2318.0,2318.0,2318.0,2318.0
202006,NaN,NaN,NaN,NaN,NaN,2463.0,2463.0,2463.0,2463.0,2463.0,...,2463.0,2463.0,2463.0,2463.0,2463.0,2463.0,2463.0,2463.0,2463.0,2463.0
202007,NaN,NaN,NaN,NaN,NaN,NaN,1536.0,1536.0,1536.0,1536.0,...,1536.0,1536.0,1536.0,1536.0,1536.0,1536.0,1536.0,1536.0,1536.0,1536.0
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2653.0,2653.0,2653.0,...,2653.0,2653.0,2653.0,2653.0,2653.0,2653.0,2653.0,2653.0,2653.0,2653.0
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2443.0,2443.0,...,2443.0,2443.0,2443.0,2443.0,2443.0,2443.0,2443.0,2443.0,2443.0,2443.0


In [7]:
attrition_cnt = pd.pivot_table(all_df_1,values='attrition_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
attrition_cnt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,2.0,2.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,6.0,7.0,8.0
202002,NaN,0.0,2.0,4.0,6.0,8.0,12.0,15.0,18.0,22.0,...,77.0,79.0,79.0,80.0,82.0,82.0,83.0,87.0,87.0,89.0
202003,NaN,NaN,0.0,5.0,8.0,10.0,13.0,23.0,30.0,35.0,...,146.0,152.0,159.0,163.0,167.0,172.0,179.0,195.0,200.0,206.0
202004,NaN,NaN,NaN,5.0,6.0,8.0,14.0,18.0,22.0,25.0,...,150.0,155.0,160.0,165.0,166.0,171.0,179.0,188.0,194.0,195.0
202005,NaN,NaN,NaN,NaN,1.0,3.0,6.0,11.0,16.0,20.0,...,121.0,125.0,128.0,128.0,132.0,135.0,139.0,146.0,152.0,153.0
202006,NaN,NaN,NaN,NaN,NaN,1.0,5.0,14.0,21.0,23.0,...,193.0,199.0,204.0,207.0,214.0,224.0,230.0,235.0,237.0,239.0
202007,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,14.0,19.0,...,193.0,199.0,201.0,206.0,210.0,216.0,217.0,225.0,229.0,230.0
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,12.0,16.0,...,317.0,331.0,342.0,347.0,352.0,357.0,365.0,376.0,380.0,387.0
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,8.0,...,250.0,267.0,272.0,276.0,284.0,294.0,300.0,308.0,316.0,320.0


In [8]:
co_cnt = pd.pivot_table(all_df_1,values='co_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
co_cnt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.0,13.0,13.0,13.0,14.0,15.0,15.0,15.0,15.0,15.0
202002,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,...,560.0,571.0,576.0,590.0,598.0,606.0,621.0,633.0,649.0,663.0
202003,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,2483.0,2537.0,2570.0,2617.0,2658.0,2696.0,2735.0,2777.0,2824.0,2876.0
202004,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1803.0,1848.0,1884.0,1920.0,1955.0,1989.0,2032.0,2067.0,2120.0,2164.0
202005,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,658.0,678.0,690.0,708.0,730.0,742.0,750.0,768.0,780.0,791.0
202006,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,588.0,615.0,635.0,652.0,681.0,693.0,712.0,724.0,741.0,762.0
202007,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,330.0,342.0,352.0,357.0,368.0,380.0,399.0,413.0,425.0,441.0
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,434.0,458.0,472.0,493.0,513.0,534.0,557.0,580.0,615.0,629.0
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,354.0,375.0,385.0,404.0,425.0,441.0,454.0,477.0,497.0,521.0


In [9]:
other_close_cnt = pd.pivot_table(all_df_1,values='other_close_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
other_close_cnt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
202002,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,...,63.0,63.0,65.0,66.0,67.0,68.0,68.0,68.0,69.0,69.0
202003,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,...,216.0,218.0,221.0,224.0,226.0,228.0,230.0,232.0,232.0,233.0
202004,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,12.0,...,169.0,170.0,173.0,175.0,178.0,181.0,183.0,184.0,185.0,187.0
202005,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,4.0,...,58.0,59.0,59.0,59.0,59.0,60.0,60.0,61.0,61.0,62.0
202006,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,2.0,...,65.0,66.0,69.0,69.0,71.0,72.0,72.0,73.0,73.0,75.0
202007,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,32.0,33.0,33.0,34.0,34.0,35.0,35.0,37.0,37.0,37.0
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,47.0,47.0,47.0,49.0,50.0,51.0,53.0,53.0,54.0,55.0
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,30.0,31.0,33.0,34.0,36.0,37.0,37.0,39.0,39.0,40.0


In [10]:
open_acct_cnt = pd.pivot_table(all_df_1,values='open_acct_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
open_acct_cnt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,110.0,110.0,111.0,111.0,110.0,110.0,109.0,109.0,110.0,110.0,...,94.0,93.0,93.0,93.0,92.0,91.0,91.0,90.0,89.0,88.0
202002,NaN,1742.0,1740.0,1738.0,1736.0,1734.0,1730.0,1727.0,1722.0,1712.0,...,1042.0,1029.0,1022.0,1006.0,995.0,986.0,970.0,954.0,937.0,921.0
202003,NaN,NaN,6531.0,6526.0,6524.0,6522.0,6519.0,6509.0,6502.0,6471.0,...,3687.0,3625.0,3582.0,3528.0,3481.0,3436.0,3388.0,3328.0,3276.0,3217.0
202004,NaN,NaN,NaN,5305.0,5304.0,5302.0,5297.0,5293.0,5289.0,5274.0,...,3189.0,3138.0,3094.0,3051.0,3012.0,2970.0,2917.0,2872.0,2812.0,2765.0
202005,NaN,NaN,NaN,NaN,2317.0,2315.0,2312.0,2307.0,2302.0,2294.0,...,1481.0,1456.0,1441.0,1423.0,1397.0,1381.0,1369.0,1343.0,1325.0,1312.0
202006,NaN,NaN,NaN,NaN,NaN,2462.0,2458.0,2449.0,2442.0,2438.0,...,1617.0,1583.0,1555.0,1535.0,1497.0,1474.0,1449.0,1431.0,1412.0,1387.0
202007,NaN,NaN,NaN,NaN,NaN,NaN,1532.0,1531.0,1522.0,1517.0,...,981.0,962.0,950.0,939.0,924.0,905.0,885.0,861.0,845.0,828.0
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2649.0,2641.0,2637.0,...,1855.0,1817.0,1792.0,1764.0,1738.0,1711.0,1678.0,1644.0,1604.0,1582.0
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2438.0,2435.0,...,1809.0,1770.0,1753.0,1729.0,1698.0,1671.0,1652.0,1619.0,1591.0,1562.0


In [11]:
all_df_1['retention_pct'] = all_df_1['open_acct_cnt']/ all_df_1['actived_acct_cnt']
retention_pct = pd.pivot_table(all_df_1,values='retention_pct',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
retention_pct.iloc[-1] = open_acct_cnt.iloc[-1]/actived_acct_cnt.iloc[-1]
retention_pct

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,1.0,1.0,1.000000,1.000000,0.990991,0.990991,0.981982,0.981982,0.982143,0.982143,...,0.831858,0.823009,0.823009,0.823009,0.814159,0.805310,0.805310,0.796460,0.787611,0.778761
202002,NaN,1.0,0.998852,0.997704,0.996556,0.995408,0.993111,0.991389,0.988519,0.982778,...,0.598163,0.590700,0.586682,0.577497,0.571183,0.566016,0.556831,0.547646,0.537887,0.528703
202003,NaN,NaN,1.000000,0.999234,0.998775,0.998469,0.998010,0.996479,0.995407,0.990661,...,0.564452,0.554960,0.548377,0.540110,0.532915,0.526026,0.518677,0.509492,0.501531,0.492498
202004,NaN,NaN,NaN,0.999058,0.998870,0.998493,0.997364,0.996611,0.995858,0.993033,...,0.600452,0.590849,0.582564,0.574468,0.567125,0.559217,0.549237,0.540764,0.529467,0.520618
202005,NaN,NaN,NaN,NaN,0.999569,0.998706,0.997412,0.995255,0.993097,0.989646,...,0.638913,0.628128,0.621657,0.613891,0.602675,0.595772,0.590595,0.579379,0.571613,0.566005
202006,NaN,NaN,NaN,NaN,NaN,0.999594,0.997970,0.994316,0.991474,0.989850,...,0.656516,0.642712,0.631344,0.623224,0.607795,0.598457,0.588307,0.580999,0.573285,0.563134
202007,NaN,NaN,NaN,NaN,NaN,NaN,0.997396,0.996745,0.990885,0.987630,...,0.638672,0.626302,0.618490,0.611328,0.601562,0.589193,0.576172,0.560547,0.550130,0.539062
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.998492,0.995477,0.993969,...,0.699208,0.684885,0.675462,0.664908,0.655107,0.644930,0.632492,0.619676,0.604599,0.596306
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.997953,0.996725,...,0.740483,0.724519,0.717560,0.707736,0.695047,0.683995,0.676218,0.662710,0.651248,0.639378


In [12]:
actived_credit_limit = pd.pivot_table(all_df_1,values='actived_credit_limit',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
actived_credit_limit

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,454500.0,459500.0,471500.0,471500.0,550500.0,595500.0,604000.0,604000.0,606000.0,606000.0,...,7.856000e+05,7.896000e+05,7.896000e+05,7.906000e+05,7.911000e+05,8.066000e+05,8.266000e+05,8.236000e+05,8.136000e+05,8.036000e+05
202002,NaN,3439000.0,3435000.0,3424000.0,3415000.0,3426000.0,3561000.0,3556000.0,3540000.0,3526000.0,...,4.264500e+06,4.299500e+06,4.321510e+06,4.319040e+06,4.305550e+06,4.380580e+06,4.421590e+06,4.411090e+06,4.404090e+06,4.415090e+06
202003,NaN,NaN,10897000.0,10891000.0,10876000.0,10863000.0,11158000.0,11126000.0,11107500.0,11065500.0,...,1.367850e+07,1.379800e+07,1.383768e+07,1.381418e+07,1.379818e+07,1.418818e+07,1.416518e+07,1.437018e+07,1.433468e+07,1.429768e+07
202004,NaN,NaN,NaN,10569500.0,10566500.0,10562500.0,10619500.0,10614000.0,10615500.0,10587000.0,...,1.285850e+07,1.292000e+07,1.298529e+07,1.296329e+07,1.294679e+07,1.333229e+07,1.330479e+07,1.326629e+07,1.333729e+07,1.332129e+07
202005,NaN,NaN,NaN,NaN,5107500.0,5103500.0,5118500.0,5111500.0,5141500.0,5115500.0,...,6.290010e+06,6.298510e+06,6.352760e+06,6.355760e+06,6.351760e+06,6.584760e+06,6.657760e+06,6.625260e+06,6.602260e+06,6.648760e+06
202006,NaN,NaN,NaN,NaN,NaN,5587500.0,5595500.0,5568500.0,5779000.0,5773000.0,...,6.933000e+06,6.925000e+06,6.968800e+06,6.960300e+06,6.941800e+06,7.112800e+06,7.096800e+06,7.157300e+06,7.134300e+06,7.114300e+06
202007,NaN,NaN,NaN,NaN,NaN,NaN,3689000.0,3684000.0,3654000.0,3647000.0,...,4.155500e+06,4.126000e+06,4.126500e+06,4.110000e+06,4.090500e+06,4.243010e+06,4.233510e+06,4.201010e+06,4.216510e+06,4.208510e+06
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4784000.0,4766000.0,4756000.0,...,6.031000e+06,5.989000e+06,6.010600e+06,5.996100e+06,5.977600e+06,6.334600e+06,6.385600e+06,6.346100e+06,6.329100e+06,6.337600e+06
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3787500.0,3783500.0,...,5.270510e+06,5.210010e+06,5.259470e+06,5.259480e+06,5.243480e+06,5.623980e+06,5.607480e+06,5.642980e+06,5.610480e+06,5.578980e+06


In [13]:
open_credit_limit = pd.pivot_table(all_df_1,values='open_credit_limit',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
open_credit_limit

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,454500.0,459500.0,471500.0,471500.0,550500.0,595500.0,604000.0,604000.0,606000.0,606000.0,...,7.571000e+05,7.601000e+05,7.601000e+05,7.611000e+05,7.606000e+05,7.731000e+05,7.931000e+05,7.901000e+05,7.801000e+05,7.701000e+05
202002,NaN,3439000.0,3435000.0,3424000.0,3415000.0,3426000.0,3561000.0,3556000.0,3539000.0,3524000.0,...,3.392500e+06,3.422000e+06,3.442010e+06,3.411540e+06,3.389550e+06,3.449080e+06,3.468090e+06,3.433090e+06,3.394090e+06,3.363590e+06
202003,NaN,NaN,10897000.0,10891000.0,10876000.0,10863000.0,11158000.0,11126000.0,11107500.0,11064500.0,...,1.019200e+07,1.022500e+07,1.022218e+07,1.011618e+07,1.003618e+07,1.036168e+07,1.023618e+07,1.035118e+07,1.019218e+07,1.004168e+07
202004,NaN,NaN,NaN,10569500.0,10566500.0,10562500.0,10619500.0,10614000.0,10615500.0,10587000.0,...,9.770500e+06,9.777000e+06,9.758790e+06,9.659790e+06,9.587790e+06,9.896290e+06,9.763790e+06,9.628790e+06,9.575290e+06,9.423790e+06
202005,NaN,NaN,NaN,NaN,5107500.0,5103500.0,5118500.0,5111500.0,5141500.0,5115500.0,...,4.989510e+06,4.975010e+06,5.003260e+06,4.976760e+06,4.912760e+06,5.119760e+06,5.163760e+06,5.073760e+06,5.024760e+06,5.045260e+06
202006,NaN,NaN,NaN,NaN,NaN,5587500.0,5595500.0,5568500.0,5779000.0,5773000.0,...,5.746500e+06,5.702000e+06,5.696800e+06,5.656800e+06,5.546800e+06,5.706800e+06,5.644300e+06,5.685300e+06,5.607800e+06,5.539800e+06
202007,NaN,NaN,NaN,NaN,NaN,NaN,3689000.0,3684000.0,3654000.0,3647000.0,...,3.400500e+06,3.371000e+06,3.349000e+06,3.330500e+06,3.288500e+06,3.399510e+06,3.338510e+06,3.260510e+06,3.220010e+06,3.155010e+06
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4784000.0,4766000.0,4756000.0,...,5.271500e+06,5.187000e+06,5.191600e+06,5.152100e+06,5.090600e+06,5.406600e+06,5.400100e+06,5.305600e+06,5.205100e+06,5.188100e+06
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3787500.0,3783500.0,...,4.681510e+06,4.634510e+06,4.663470e+06,4.618480e+06,4.557980e+06,4.910980e+06,4.872980e+06,4.866480e+06,4.800480e+06,4.734980e+06


In [14]:
active_credit_limit = pd.pivot_table(all_df_1,values='active_credit_limit',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
active_credit_limit

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,350500.0,407500.0,401000.0,365500.0,310000.0,421000.0,458500.0,400500.0,512500.0,505500.0,...,5.375000e+05,4.055000e+05,5.265000e+05,5.165000e+05,5.775000e+05,5.580000e+05,5.115000e+05,5.565000e+05,5.660000e+05,4.525000e+05
202002,NaN,2695000.0,3125500.0,2963500.0,2991500.0,2911500.0,3058500.0,3022500.0,3051500.0,3007500.0,...,2.717500e+06,2.796000e+06,2.727520e+06,2.712550e+06,2.682560e+06,2.755330e+06,2.763100e+06,2.735600e+06,2.707100e+06,2.689600e+06
202003,NaN,NaN,7678000.0,9806500.0,9682000.0,9676000.0,9930000.0,9884000.0,9957500.0,9792000.0,...,8.547000e+06,8.668000e+06,8.510300e+06,8.430300e+06,8.270800e+06,8.684800e+06,8.493800e+06,8.700300e+06,8.526175e+06,8.326500e+06
202004,NaN,NaN,NaN,8085500.0,9140500.0,9180500.0,9264500.0,9391000.0,9501000.0,9320000.0,...,8.191500e+06,8.325000e+06,8.208750e+06,7.999750e+06,7.998500e+06,8.238000e+06,8.116000e+06,8.038500e+06,7.958500e+06,7.818500e+06
202005,NaN,NaN,NaN,NaN,4035500.0,4387000.0,4395000.0,4403000.0,4540500.0,4470000.0,...,4.050000e+06,4.091010e+06,4.111010e+06,4.059510e+06,3.967500e+06,4.246000e+06,4.254760e+06,4.159000e+06,4.112010e+06,4.156010e+06
202006,NaN,NaN,NaN,NaN,NaN,4406000.0,5077500.0,5004000.0,5281500.0,5145500.0,...,4.908000e+06,4.903000e+06,4.803000e+06,4.711500e+06,4.596000e+06,4.785000e+06,4.771800e+06,4.814800e+06,4.761300e+06,4.673800e+06
202007,NaN,NaN,NaN,NaN,NaN,NaN,2723000.0,3341000.0,3321500.0,3246500.0,...,2.897500e+06,2.858500e+06,2.826000e+06,2.788000e+06,2.787000e+06,2.874010e+06,2.812010e+06,2.733510e+06,2.720510e+06,2.675510e+06
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3842000.0,4379000.0,4280500.0,...,4.472000e+06,4.430500e+06,4.395500e+06,4.285000e+06,4.216500e+06,4.564000e+06,4.471500e+06,4.443500e+06,4.354500e+06,4.336500e+06
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3033000.0,3457000.0,...,3.960510e+06,3.927010e+06,3.866510e+06,3.794520e+06,3.677980e+06,4.081520e+06,4.027020e+06,4.038020e+06,3.958020e+06,3.856520e+06


In [16]:
avg_active_credit_line = pd.pivot_table(all_df_1,values='avg_active_credit_line',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
active_acct_cnt = pd.pivot_table(all_df_1,values='active_acct_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
avg_active_credit_line.iloc[-1] = active_credit_limit.iloc[-1]/active_acct_cnt.iloc[-1]
avg_active_credit_line

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,4736.490000,4158.160000,4406.590000,4250.000000,4078.950000,5072.290000,5151.690000,4767.860000,5694.440000,5679.780000,...,8398.440000,6991.380000,8491.940000,8608.330000,9467.210000,9147.540000,8669.490000,9594.830000,9929.820000,8227.270000
202002,NaN,1962.860000,1925.750000,1897.250000,1893.350000,1866.350000,1965.620000,1942.480000,1941.160000,1934.080000,...,3148.900000,3228.640000,3247.050000,3287.940000,3291.480000,3431.300000,3471.230000,3489.290000,3529.470000,3586.130000
202003,NaN,NaN,1583.750000,1630.340000,1623.950000,1616.440000,1661.090000,1649.810000,1652.150000,1651.540000,...,2733.290000,2806.090000,2820.780000,2830.860000,2841.220000,3015.560000,3006.650000,3110.580000,3109.470000,3111.550000
202004,NaN,NaN,NaN,1886.490000,1927.160000,1908.230000,1917.720000,1929.530000,1942.150000,1930.410000,...,3008.260000,3075.360000,3099.980000,3095.880000,3129.300000,3288.620000,3308.600000,3314.850000,3367.960000,3364.240000
202005,NaN,NaN,NaN,NaN,2070.550000,2115.240000,2124.210000,2118.860000,2163.170000,2143.880000,...,3327.860000,3361.550000,3420.140000,3440.260000,3450.000000,3685.760000,3738.800000,3763.800000,3769.030000,3862.460000
202006,NaN,NaN,NaN,NaN,NaN,2192.040000,2215.310000,2205.380000,2303.310000,2295.050000,...,3553.950000,3607.800000,3630.390000,3624.230000,3665.070000,3880.780000,3898.530000,3982.460000,3971.060000,4015.290000
202007,NaN,NaN,NaN,NaN,NaN,NaN,2367.830000,2356.140000,2339.080000,2355.950000,...,3461.770000,3503.060000,3501.860000,3560.660000,3605.430000,3821.820000,3774.510000,3791.280000,3826.320000,3838.610000
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1821.720000,1767.860000,1767.340000,...,2903.900000,2909.060000,2936.210000,2938.960000,2977.750000,3255.350000,3297.570000,3323.490000,3336.780000,3417.260000
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1526.420000,1537.130000,...,2661.630000,2727.090000,2709.540000,2735.780000,2740.670000,3061.910000,3055.400000,3113.350000,3138.790000,3171.480000


In [15]:
active_acct_cnt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,74.0,98.0,91.0,86.0,76.0,83.0,89.0,84.0,90.0,89.0,...,64.0,58.0,62.0,60.0,61.0,61.0,59.0,58.0,57.0,55.0
202002,NaN,1373.0,1623.0,1562.0,1580.0,1560.0,1556.0,1556.0,1572.0,1555.0,...,863.0,866.0,840.0,825.0,815.0,803.0,796.0,784.0,767.0,750.0
202003,NaN,NaN,4848.0,6015.0,5962.0,5986.0,5978.0,5991.0,6027.0,5929.0,...,3127.0,3089.0,3017.0,2978.0,2911.0,2880.0,2825.0,2797.0,2742.0,2676.0
202004,NaN,NaN,NaN,4286.0,4743.0,4811.0,4831.0,4867.0,4892.0,4828.0,...,2723.0,2707.0,2648.0,2584.0,2556.0,2505.0,2453.0,2425.0,2363.0,2324.0
202005,NaN,NaN,NaN,NaN,1949.0,2074.0,2069.0,2078.0,2099.0,2085.0,...,1217.0,1217.0,1202.0,1180.0,1150.0,1152.0,1138.0,1105.0,1091.0,1076.0
202006,NaN,NaN,NaN,NaN,NaN,2010.0,2292.0,2269.0,2293.0,2242.0,...,1381.0,1359.0,1323.0,1300.0,1254.0,1233.0,1224.0,1209.0,1199.0,1164.0
202007,NaN,NaN,NaN,NaN,NaN,NaN,1150.0,1418.0,1420.0,1378.0,...,837.0,816.0,807.0,783.0,773.0,752.0,745.0,721.0,711.0,697.0
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2109.0,2477.0,2422.0,...,1540.0,1523.0,1497.0,1458.0,1416.0,1402.0,1356.0,1337.0,1305.0,1269.0
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1987.0,2249.0,...,1488.0,1440.0,1427.0,1387.0,1342.0,1333.0,1318.0,1297.0,1261.0,1216.0


In [17]:
active_rate = pd.pivot_table(all_df_1,values='active_rate',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
active_rate.iloc[-1] = active_acct_cnt.iloc[-1]/open_acct_cnt.iloc[-1]
active_rate

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,0.672700,0.890900,0.819800,0.774800,0.690900,0.754500,0.816500,0.770600,0.818200,0.80910,...,0.680900,0.623700,0.666700,0.645200,0.663000,0.670300,0.648400,0.64440,0.640400,0.625000
202002,NaN,0.788200,0.932800,0.898700,0.910100,0.899700,0.899400,0.901000,0.912900,0.90830,...,0.828200,0.841600,0.821900,0.820100,0.819100,0.814400,0.820600,0.82180,0.818600,0.814300
202003,NaN,NaN,0.742300,0.921700,0.913900,0.917800,0.917000,0.920400,0.926900,0.91620,...,0.848100,0.852100,0.842300,0.844100,0.836300,0.838200,0.833800,0.84040,0.837000,0.831800
202004,NaN,NaN,NaN,0.807900,0.894200,0.907400,0.912000,0.919500,0.924900,0.91540,...,0.853900,0.862700,0.855900,0.846900,0.848600,0.843400,0.840900,0.84440,0.840300,0.840500
202005,NaN,NaN,NaN,NaN,0.841200,0.895900,0.894900,0.900700,0.911800,0.90890,...,0.821700,0.835900,0.834100,0.829200,0.823200,0.834200,0.831300,0.82280,0.823400,0.820100
202006,NaN,NaN,NaN,NaN,NaN,0.816400,0.932500,0.926500,0.939000,0.91960,...,0.854100,0.858500,0.850800,0.846900,0.837700,0.836500,0.844700,0.84490,0.849200,0.839200
202007,NaN,NaN,NaN,NaN,NaN,NaN,0.750700,0.926200,0.933000,0.90840,...,0.853200,0.848200,0.849500,0.833900,0.836600,0.830900,0.841800,0.83740,0.841400,0.841800
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.796100,0.937900,0.91850,...,0.830200,0.838200,0.835400,0.826500,0.814700,0.819400,0.808100,0.81330,0.813600,0.802100
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.815000,0.92360,...,0.822600,0.813600,0.814000,0.802200,0.790300,0.797700,0.797800,0.80110,0.792600,0.778500


In [18]:
transactor_acct_cnt = pd.pivot_table(all_df_1,values='transactor_acct_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
transactor_acct_cnt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,74.0,95.0,87.0,77.0,54.0,67.0,74.0,73.0,75.0,75.0,...,54.0,51.0,52.0,51.0,52.0,53.0,51.0,51.0,47.0,46.0
202002,NaN,1373.0,1473.0,1197.0,1030.0,1040.0,975.0,957.0,1002.0,944.0,...,644.0,674.0,614.0,599.0,566.0,569.0,583.0,552.0,543.0,525.0
202003,NaN,NaN,4839.0,5159.0,3851.0,3851.0,3602.0,3504.0,3469.0,3150.0,...,2290.0,2368.0,2175.0,2120.0,2021.0,2000.0,1933.0,1903.0,1818.0,1718.0
202004,NaN,NaN,NaN,4283.0,3891.0,3587.0,3272.0,3142.0,3099.0,2838.0,...,1974.0,1995.0,1879.0,1777.0,1721.0,1710.0,1684.0,1595.0,1566.0,1480.0
202005,NaN,NaN,NaN,NaN,1948.0,1781.0,1580.0,1487.0,1466.0,1366.0,...,889.0,945.0,890.0,891.0,792.0,858.0,819.0,785.0,753.0,720.0
202006,NaN,NaN,NaN,NaN,NaN,2010.0,2089.0,1882.0,1817.0,1651.0,...,1025.0,1032.0,985.0,956.0,889.0,917.0,899.0,835.0,829.0,755.0
202007,NaN,NaN,NaN,NaN,NaN,NaN,1148.0,1337.0,1230.0,1105.0,...,661.0,646.0,605.0,567.0,566.0,545.0,531.0,513.0,507.0,496.0
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2106.0,2334.0,2110.0,...,1217.0,1215.0,1145.0,1054.0,1029.0,1049.0,1002.0,947.0,935.0,889.0
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1984.0,2136.0,...,1183.0,1146.0,1100.0,1035.0,997.0,1014.0,977.0,927.0,919.0,858.0


In [20]:
transactor_acct_pct = pd.pivot_table(all_df_1,values='transactor_acct_pct',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
transactor_acct_pct.iloc[-1] =  transactor_acct_cnt.iloc[-1]/open_acct_cnt.iloc[-1]
transactor_acct_pct

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
202002,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
202003,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
202004,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
202005,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
202006,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
202007,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
revolver_acct_cnt = pd.pivot_table(all_df_1,values='revolver_acct_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
revolver_acct_cnt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,0.0,0.0,0.0,26.0,26.0,34.0,31.0,31.0,31.0,20.0,...,19.0,20.0,22.0,21.0,23.0,19.0,19.0,21.0,21.0,19.0
202002,NaN,0.0,0.0,787.0,854.0,924.0,916.0,914.0,953.0,602.0,...,453.0,477.0,486.0,473.0,477.0,425.0,446.0,433.0,437.0,411.0
202003,NaN,NaN,0.0,2283.0,3410.0,3684.0,3701.0,3769.0,3914.0,2510.0,...,1748.0,1762.0,1826.0,1767.0,1743.0,1714.0,1724.0,1732.0,1732.0,1659.0
202004,NaN,NaN,NaN,0.0,2020.0,2587.0,2716.0,2821.0,2971.0,2189.0,...,1582.0,1635.0,1671.0,1603.0,1605.0,1526.0,1539.0,1521.0,1506.0,1427.0
202005,NaN,NaN,NaN,NaN,0.0,789.0,960.0,1025.0,1147.0,947.0,...,668.0,672.0,701.0,677.0,704.0,670.0,690.0,688.0,666.0,629.0
202006,NaN,NaN,NaN,NaN,NaN,0.0,626.0,868.0,1025.0,940.0,...,719.0,740.0,737.0,715.0,721.0,704.0,744.0,743.0,719.0,709.0
202007,NaN,NaN,NaN,NaN,NaN,NaN,1.0,325.0,513.0,562.0,...,430.0,449.0,452.0,420.0,415.0,406.0,407.0,404.0,386.0,373.0
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,494.0,731.0,...,708.0,755.0,757.0,737.0,732.0,685.0,730.0,757.0,714.0,687.0
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,370.0,...,688.0,701.0,727.0,691.0,677.0,660.0,679.0,687.0,675.0,647.0


In [22]:
revolver_acct_pct = pd.pivot_table(all_df_1,values='revolver_acct_pct',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
revolver_acct_pct.iloc[-1] = revolver_acct_cnt.iloc[-1]/open_acct_cnt.iloc[-1]
revolver_acct_pct

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
202002,NaN,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
202003,NaN,NaN,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
202004,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
202005,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
202006,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
202007,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [23]:
tot_end_os_bal_amt = pd.pivot_table(all_df_1,values='tot_end_os_bal_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
tot_end_os_bal_amt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,134447.22,186610.78,171485.03,162465.64,122620.37,1.413684e+05,1.563188e+05,1.563173e+05,1.708283e+05,173097.02,...,2.146955e+05,2.167690e+05,1.927538e+05,2.264541e+05,1.971050e+05,1.599579e+05,1.737714e+05,2.017078e+05,2.187269e+05,1.921946e+05
202002,NaN,1550453.86,2059454.59,2030775.73,2080715.03,2.261394e+06,2.344730e+06,2.521644e+06,2.721647e+06,2676659.97,...,1.862261e+06,2.025144e+06,1.952063e+06,2.028904e+06,2.010687e+06,2.061234e+06,2.137686e+06,2.086245e+06,2.028403e+06,2.036218e+06
202003,NaN,NaN,4803459.51,7135948.67,7077591.01,7.872173e+06,8.360309e+06,8.909798e+06,9.630270e+06,9388303.04,...,6.167623e+06,6.537538e+06,6.473398e+06,6.494284e+06,6.349151e+06,6.609738e+06,6.742386e+06,6.881542e+06,6.930454e+06,6.646903e+06
202004,NaN,NaN,NaN,5146364.67,6252959.32,6.885016e+06,7.363974e+06,7.765031e+06,8.456298e+06,8273917.37,...,6.109175e+06,6.414880e+06,6.330220e+06,6.279171e+06,6.137833e+06,6.428021e+06,6.521481e+06,6.475050e+06,6.533618e+06,6.284800e+06
202005,NaN,NaN,NaN,NaN,2499780.65,2.995533e+06,3.165027e+06,3.360157e+06,2.911452e+06,3585636.24,...,2.861732e+06,3.008397e+06,3.038727e+06,3.073200e+06,2.952511e+06,3.140482e+06,3.244497e+06,3.175324e+06,3.312911e+06,3.224057e+06
202006,NaN,NaN,NaN,NaN,NaN,2.590723e+06,3.303748e+06,3.425964e+06,4.003034e+06,3911831.93,...,3.486261e+06,3.516965e+06,3.504489e+06,3.516282e+06,3.382541e+06,3.576476e+06,3.649612e+06,3.657654e+06,3.709077e+06,3.617845e+06
202007,NaN,NaN,NaN,NaN,NaN,NaN,1.528846e+06,2.196745e+06,2.448749e+06,2341006.74,...,2.142543e+06,2.167962e+06,2.159794e+06,2.157478e+06,2.165543e+06,2.278579e+06,2.255241e+06,2.182960e+06,2.196418e+06,2.111589e+06
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.305087e+06,3.052492e+06,2996335.57,...,3.057081e+06,3.171003e+06,3.153789e+06,3.165689e+06,3.076707e+06,3.309817e+06,3.424836e+06,3.463117e+06,3.383381e+06,3.258855e+06
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.945252e+06,2275743.96,...,2.742575e+06,2.889966e+06,2.842182e+06,2.888777e+06,2.763139e+06,3.091880e+06,3.111519e+06,3.192548e+06,3.180336e+06,3.053222e+06


In [24]:
acct_end_os_bal_amt = pd.pivot_table(all_df_1,values='acct_end_os_bal_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
acct_end_os_bal_amt.iloc[-1] = tot_end_os_bal_amt.iloc[-1]/active_acct_cnt.iloc[-1]
acct_end_os_bal_amt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,1816.850000,1904.190000,1884.450000,1889.140000,1613.430000,1703.230000,1756.390000,1860.92000,1898.090000,1944.910000,...,3354.620000,3737.400000,3108.930000,3774.230000,3231.230000,2622.260000,2945.280000,3477.720000,3837.310000,3494.450000
202002,NaN,1129.250000,1268.920000,1300.110000,1316.910000,1449.610000,1506.900000,1620.59000,1731.330000,1721.320000,...,2157.890000,2338.500000,2323.880000,2459.280000,2467.100000,2566.920000,2685.540000,2661.030000,2644.590000,2714.960000
202003,NaN,NaN,990.810000,1186.360000,1187.120000,1315.100000,1398.510000,1487.20000,1597.850000,1583.450000,...,1972.380000,2116.390000,2145.640000,2180.750000,2181.090000,2295.050000,2386.690000,2460.330000,2527.520000,2483.900000
202004,NaN,NaN,NaN,1200.740000,1318.360000,1431.100000,1524.320000,1595.45000,1728.600000,1713.740000,...,2243.550000,2369.740000,2390.570000,2430.020000,2401.340000,2566.080000,2658.570000,2670.120000,2764.970000,2704.300000
202005,NaN,NaN,NaN,NaN,1282.600000,1444.330000,1529.740000,1617.01000,1387.070000,1719.730000,...,2351.460000,2471.980000,2528.060000,2604.410000,2567.400000,2726.110000,2851.050000,2873.600000,3036.580000,2996.340000
202006,NaN,NaN,NaN,NaN,NaN,1288.920000,1441.430000,1509.90000,1745.760000,1744.800000,...,2524.450000,2587.910000,2648.900000,2704.830000,2697.400000,2900.630000,2981.710000,3025.360000,3093.480000,3108.110000
202007,NaN,NaN,NaN,NaN,NaN,NaN,1329.430000,1549.19000,1724.470000,1698.840000,...,2559.790000,2656.820000,2676.320000,2755.400000,2801.480000,3030.020000,3027.170000,3027.680000,3089.200000,3029.540000
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1092.98000,1232.330000,1237.130000,...,1985.120000,2082.080000,2106.740000,2171.250000,2172.820000,2360.780000,2525.690000,2590.210000,2592.630000,2568.050000
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,978.990000,1011.890000,...,1843.130000,2006.920000,1991.720000,2082.750000,2058.970000,2319.490000,2360.790000,2461.490000,2522.070000,2510.870000


In [27]:
tot_txn_amt = pd.pivot_table(all_df_1,values='tot_txn_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
tot_txn_amt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,145655.06,200992.55,177305.50,147278.24,85526.36,117567.36,116033.21,137776.83,148254.81,144452.45,...,2.172901e+05,2.070590e+05,1.950082e+05,1.822694e+05,1.435842e+05,1.367999e+05,1.490291e+05,1.859234e+05,1.473627e+05,1.568162e+05
202002,NaN,1698875.90,1859105.79,1116273.55,961735.49,1054031.07,1011941.56,1109130.25,1156376.29,1086261.40,...,1.145995e+06,1.287528e+06,1.043314e+06,1.092850e+06,9.477622e+05,1.138113e+06,1.148905e+06,1.037716e+06,1.001297e+06,1.070811e+06
202003,NaN,NaN,4991544.81,5411258.85,3186830.78,3580748.63,3379950.15,3464417.63,3494990.43,3027330.48,...,3.436461e+06,3.557642e+06,3.015267e+06,2.955510e+06,2.679083e+06,3.080229e+06,2.887093e+06,3.029091e+06,2.808902e+06,2.649003e+06
202004,NaN,NaN,NaN,5379931.02,4191991.22,3855690.36,3536255.65,3399581.21,3440384.03,2914933.57,...,3.171246e+06,3.061407e+06,2.698713e+06,2.567493e+06,2.284954e+06,2.865820e+06,2.672762e+06,2.504278e+06,2.595393e+06,2.355595e+06
202005,NaN,NaN,NaN,NaN,2711326.06,2254393.75,1923763.97,1884392.19,2074276.28,1572376.83,...,1.670422e+06,1.677296e+06,1.562764e+06,1.454287e+06,1.277777e+06,1.607520e+06,1.610014e+06,1.442655e+06,1.503659e+06,1.338799e+06
202006,NaN,NaN,NaN,NaN,NaN,2983232.90,3103164.58,2616174.98,2854637.82,2408841.61,...,2.024601e+06,1.982319e+06,1.908154e+06,1.794636e+06,1.639673e+06,1.825526e+06,1.781969e+06,1.743221e+06,1.871436e+06,1.689949e+06
202007,NaN,NaN,NaN,NaN,NaN,NaN,1742796.93,2357799.81,1968211.82,1662558.21,...,1.306387e+06,1.317655e+06,1.231083e+06,1.057209e+06,1.030346e+06,1.183941e+06,1.091023e+06,1.040939e+06,1.069126e+06,9.994959e+05
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2921228.68,3369359.10,2747507.13,...,2.098852e+06,2.053687e+06,1.837229e+06,1.791080e+06,1.659066e+06,2.055879e+06,1.866856e+06,1.690787e+06,1.715734e+06,1.597663e+06
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2369215.99,2733815.85,...,1.991304e+06,1.955481e+06,1.765083e+06,1.663711e+06,1.546757e+06,2.009187e+06,1.752147e+06,1.729104e+06,1.698211e+06,1.630710e+06


In [28]:
acct_txn_amt = pd.pivot_table(all_df_1,values='acct_txn_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
acct_txn_amt.iloc[-1] = tot_txn_amt.iloc[-1]/active_acct_cnt.iloc[-1]
acct_txn_amt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,1968.310000,2050.940000,1948.410000,1712.540000,1125.350000,1416.470000,1303.740000,1640.200000,1647.280000,1623.060000,...,3395.160000,3569.980000,3145.290000,3037.820000,2353.840000,2242.620000,2525.920000,3205.580000,2585.31000,2851.200000
202002,NaN,1237.350000,1145.470000,714.640000,608.690000,675.660000,650.350000,712.810000,735.610000,698.560000,...,1327.920000,1486.750000,1242.040000,1324.670000,1162.900000,1417.330000,1443.350000,1323.620000,1305.47000,1427.750000
202003,NaN,NaN,1029.610000,899.630000,534.520000,598.190000,565.400000,578.270000,579.890000,510.600000,...,1098.960000,1151.710000,999.430000,992.450000,920.330000,1069.520000,1021.980000,1082.980000,1024.40000,989.910000
202004,NaN,NaN,NaN,1255.230000,883.830000,801.430000,731.990000,698.500000,703.270000,603.760000,...,1164.610000,1130.920000,1019.150000,993.610000,893.960000,1144.040000,1089.590000,1032.690000,1098.35000,1013.600000
202005,NaN,NaN,NaN,NaN,1391.140000,1086.980000,929.800000,906.830000,988.220000,754.140000,...,1372.570000,1378.220000,1300.140000,1232.450000,1111.110000,1395.420000,1414.780000,1305.570000,1378.24000,1244.240000
202006,NaN,NaN,NaN,NaN,NaN,1484.200000,1353.910000,1153.010000,1244.940000,1074.420000,...,1466.040000,1458.660000,1442.290000,1380.490000,1307.550000,1480.560000,1455.860000,1441.870000,1560.83000,1451.850000
202007,NaN,NaN,NaN,NaN,NaN,NaN,1515.480000,1662.760000,1386.060000,1206.500000,...,1560.800000,1614.770000,1525.510000,1350.200000,1332.920000,1574.390000,1464.460000,1443.740000,1503.69000,1434.000000
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1385.130000,1360.260000,1134.400000,...,1362.890000,1348.450000,1227.270000,1228.450000,1171.660000,1466.390000,1376.740000,1264.610000,1314.74000,1258.990000
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1192.360000,1215.570000,...,1338.240000,1357.970000,1236.920000,1199.500000,1152.580000,1507.270000,1329.400000,1333.160000,1346.72000,1341.040000


In [29]:
txn_cl_util_pct = pd.pivot_table(all_df_1,values='txn_cl_util_pct',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
txn_cl_util_pct.iloc[-1] = tot_txn_amt.iloc[-1]/active_credit_limit.iloc[-1]
txn_cl_util_pct

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,0.415500,0.493200,0.442100,0.402900,0.27580,0.279200,0.253000,0.344000,0.289200,0.285700,...,0.404200,0.510600,0.370300,0.35280,0.248600,0.245100,0.291300,0.334000,0.260300,0.346500
202002,NaN,0.630300,0.594800,0.376600,0.32140,0.362000,0.330800,0.366900,0.378900,0.361100,...,0.421700,0.460400,0.382500,0.40280,0.353300,0.413000,0.415800,0.379300,0.369800,0.398100
202003,NaN,NaN,0.650100,0.551800,0.32910,0.370000,0.340300,0.350500,0.350900,0.309100,...,0.402000,0.410400,0.354300,0.35050,0.323900,0.354600,0.339900,0.348100,0.329400,0.318100
202004,NaN,NaN,NaN,0.665300,0.45860,0.419900,0.381600,0.362000,0.362100,0.312700,...,0.387100,0.367700,0.328700,0.32090,0.285600,0.347800,0.329300,0.311500,0.326100,0.301200
202005,NaN,NaN,NaN,NaN,0.67180,0.513800,0.437700,0.427900,0.456800,0.351700,...,0.412400,0.409900,0.380100,0.35820,0.322000,0.378500,0.378400,0.346800,0.365600,0.322100
202006,NaN,NaN,NaN,NaN,NaN,0.677000,0.611100,0.522800,0.540400,0.468100,...,0.412500,0.404300,0.397200,0.38090,0.356700,0.381500,0.373400,0.362000,0.393000,0.361500
202007,NaN,NaN,NaN,NaN,NaN,NaN,0.640000,0.705700,0.592500,0.512100,...,0.450800,0.460900,0.435600,0.37910,0.369600,0.411900,0.387900,0.380800,0.392900,0.373500
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.760300,0.769400,0.641800,...,0.469300,0.463500,0.417900,0.41790,0.393400,0.450400,0.417500,0.380500,0.394000,0.368400
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.781100,0.790800,...,0.502700,0.497900,0.456500,0.43840,0.420500,0.492200,0.435000,0.428200,0.429000,0.422800


In [30]:
tot_txn_cnt = pd.pivot_table(all_df_1,values='tot_txn_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
tot_txn_cnt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,513.0,887.0,695.0,409.0,265.0,345.0,364.0,466.0,475.0,387.0,...,493.0,545.0,476.0,447.0,406.0,378.0,459.0,470.0,421.0,393.0
202002,NaN,6315.0,8541.0,5201.0,3903.0,4239.0,4108.0,4368.0,4807.0,4433.0,...,3850.0,4117.0,3453.0,3462.0,2989.0,3441.0,3748.0,3225.0,3334.0,3189.0
202003,NaN,NaN,20994.0,25390.0,14263.0,15813.0,15088.0,14985.0,15842.0,13333.0,...,11952.0,12612.0,10686.0,10239.0,9330.0,10864.0,9897.0,10171.0,9546.0,8458.0
202004,NaN,NaN,NaN,20339.0,16954.0,15867.0,14714.0,13849.0,14686.0,12431.0,...,10351.0,10308.0,8904.0,8680.0,7855.0,9450.0,8528.0,8039.0,8410.0,7312.0
202005,NaN,NaN,NaN,NaN,9648.0,8598.0,7485.0,7064.0,7323.0,6322.0,...,5350.0,5347.0,5010.0,4910.0,4167.0,5257.0,5050.0,4460.0,4645.0,4054.0
202006,NaN,NaN,NaN,NaN,NaN,9888.0,11725.0,9975.0,10796.0,9091.0,...,6118.0,6050.0,5728.0,5441.0,4759.0,5286.0,4947.0,4889.0,4862.0,4325.0
202007,NaN,NaN,NaN,NaN,NaN,NaN,5765.0,8367.0,7576.0,6316.0,...,4016.0,4056.0,3571.0,3365.0,2939.0,3493.0,3268.0,3076.0,3259.0,2900.0
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10403.0,14601.0,11166.0,...,6483.0,6704.0,5854.0,5778.0,5557.0,6279.0,5807.0,5151.0,5345.0,4603.0
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9507.0,11968.0,...,6788.0,6740.0,5955.0,5461.0,5178.0,6308.0,5607.0,5356.0,5257.0,4594.0


In [31]:
acct_txn_cnt = pd.pivot_table(all_df_1,values='acct_txn_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
acct_txn_cnt.iloc[-1] = tot_txn_cnt.iloc[-1]/active_acct_cnt.iloc[-1]
acct_txn_cnt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,6.930000,9.050000,7.640000,4.76000,3.49000,4.160000,4.090000,5.550000,5.280000,4.350000,...,7.700000,9.4000,7.680000,7.45000,6.660000,6.200000,7.780000,8.100000,7.390000,7.150000
202002,NaN,4.600000,5.260000,3.33000,2.47000,2.720000,2.640000,2.810000,3.060000,2.850000,...,4.460000,4.7500,4.110000,4.20000,3.670000,4.290000,4.710000,4.110000,4.350000,4.250000
202003,NaN,NaN,4.330000,4.22000,2.39000,2.640000,2.520000,2.500000,2.630000,2.250000,...,3.820000,4.0800,3.540000,3.44000,3.210000,3.770000,3.500000,3.640000,3.480000,3.160000
202004,NaN,NaN,NaN,4.75000,3.57000,3.300000,3.050000,2.850000,3.000000,2.570000,...,3.800000,3.8100,3.360000,3.36000,3.070000,3.770000,3.480000,3.320000,3.560000,3.150000
202005,NaN,NaN,NaN,NaN,4.95000,4.150000,3.620000,3.400000,3.490000,3.030000,...,4.400000,4.3900,4.170000,4.16000,3.620000,4.560000,4.440000,4.040000,4.260000,3.770000
202006,NaN,NaN,NaN,NaN,NaN,4.920000,5.120000,4.400000,4.710000,4.050000,...,4.430000,4.4500,4.330000,4.19000,3.800000,4.290000,4.040000,4.040000,4.060000,3.720000
202007,NaN,NaN,NaN,NaN,NaN,NaN,5.010000,5.900000,5.340000,4.580000,...,4.800000,4.9700,4.430000,4.30000,3.800000,4.640000,4.390000,4.270000,4.580000,4.160000
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.930000,5.890000,4.610000,...,4.210000,4.4000,3.910000,3.96000,3.920000,4.480000,4.280000,3.850000,4.100000,3.630000
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.780000,5.320000,...,4.560000,4.6800,4.170000,3.94000,3.860000,4.730000,4.250000,4.130000,4.170000,3.780000


In [32]:
txn_size_amt = pd.pivot_table(all_df_1,values='txn_size_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
txn_size_amt.iloc[-1] = tot_txn_amt.iloc[-1]/tot_txn_cnt.iloc[-1]
txn_size_amt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,283.930000,226.600000,255.120000,360.09000,322.740000,340.770000,318.770000,295.660000,312.120000,373.260000,...,440.750000,379.920000,409.680000,407.760000,353.660000,361.90000,324.680000,395.580000,350.03000,399.020000
202002,NaN,269.020000,217.670000,214.63000,246.410000,248.650000,246.330000,253.920000,240.560000,245.040000,...,297.660000,312.730000,302.150000,315.670000,317.080000,330.75000,306.540000,321.770000,300.33000,335.780000
202003,NaN,NaN,237.760000,213.13000,223.430000,226.440000,224.020000,231.190000,220.620000,227.060000,...,287.520000,282.080000,282.170000,288.650000,287.150000,283.53000,291.710000,297.820000,294.25000,313.190000
202004,NaN,NaN,NaN,264.51000,247.260000,243.000000,240.330000,245.470000,234.260000,234.490000,...,306.370000,296.990000,303.090000,295.790000,290.890000,303.26000,313.410000,311.520000,308.61000,322.150000
202005,NaN,NaN,NaN,NaN,281.020000,262.200000,257.020000,266.760000,283.250000,248.720000,...,312.230000,313.690000,311.930000,296.190000,306.640000,305.79000,318.810000,323.470000,323.72000,330.240000
202006,NaN,NaN,NaN,NaN,NaN,301.700000,264.660000,262.270000,264.420000,264.970000,...,330.930000,327.660000,333.130000,329.840000,344.540000,345.35000,360.210000,356.560000,384.91000,390.740000
202007,NaN,NaN,NaN,NaN,NaN,NaN,302.310000,281.800000,259.800000,263.230000,...,325.300000,324.870000,344.740000,314.180000,350.580000,338.95000,333.850000,338.410000,328.05000,344.650000
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280.810000,230.760000,246.060000,...,323.750000,306.340000,313.840000,309.980000,298.550000,327.42000,321.480000,328.240000,321.00000,347.090000
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,249.210000,228.430000,...,293.360000,290.130000,296.400000,304.650000,298.720000,318.51000,312.490000,322.840000,323.04000,354.970000


In [33]:
tot_adb_amt = pd.pivot_table(all_df_1,values='tot_adb_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
tot_adb_amt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,0.0,0.0,0.0,61200.71,60534.93,62941.85,66740.11,70276.44,63135.93,43735.64,...,8.454295e+04,1.068440e+05,1.274143e+05,1.256154e+05,1.275021e+05,8.899819e+04,8.219136e+04,9.272358e+04,1.277695e+05,8.591728e+04
202002,NaN,0.0,0.0,1195890.53,1285928.27,1352829.35,1447203.59,1509257.86,1613300.60,986823.84,...,1.032778e+06,1.176425e+06,1.192023e+06,1.212847e+06,1.266683e+06,1.170050e+06,1.232448e+06,1.203629e+06,1.215343e+06,1.167289e+06
202003,NaN,NaN,0.0,2856689.33,4463124.60,4852465.36,5168483.12,5516930.29,5860319.99,3495894.47,...,3.547887e+06,3.884267e+06,4.145721e+06,4.055957e+06,4.022860e+06,4.020478e+06,4.173783e+06,4.309690e+06,4.416764e+06,4.295159e+06
202004,NaN,NaN,NaN,0.00,3059199.30,3984763.67,4399469.31,4721336.83,5109939.47,3608361.52,...,3.596536e+06,4.014113e+06,4.143859e+06,4.024534e+06,4.060709e+06,3.916549e+06,4.054120e+06,4.118818e+06,4.107919e+06,3.940237e+06
202005,NaN,NaN,NaN,NaN,0.00,1304302.75,1682902.87,1841573.65,2096945.08,1715862.44,...,1.625599e+06,1.743221e+06,1.838340e+06,1.807628e+06,1.910973e+06,1.847819e+06,2.011544e+06,2.014359e+06,2.006156e+06,1.948137e+06
202006,NaN,NaN,NaN,NaN,NaN,0.00,1127647.88,1631020.08,1923770.88,1860885.57,...,1.949441e+06,2.075886e+06,2.117082e+06,2.119254e+06,2.170210e+06,2.174397e+06,2.347012e+06,2.432418e+06,2.367715e+06,2.322890e+06
202007,NaN,NaN,NaN,NaN,NaN,NaN,785.39,634764.74,1083771.95,1194570.19,...,1.211073e+06,1.294489e+06,1.323875e+06,1.268605e+06,1.222144e+06,1.288391e+06,1.288595e+06,1.248410e+06,1.231815e+06,1.227029e+06
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,756991.64,1163258.73,...,1.551180e+06,1.763912e+06,1.809472e+06,1.773427e+06,1.766406e+06,1.728080e+06,1.961197e+06,2.080797e+06,1.987181e+06,1.906041e+06
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,526648.27,...,1.364445e+06,1.534289e+06,1.635718e+06,1.587176e+06,1.597033e+06,1.641322e+06,1.748072e+06,1.822314e+06,1.831172e+06,1.767851e+06


In [34]:
acct_adb_amt = pd.pivot_table(all_df_1,values='acct_adb_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
acct_adb_amt.iloc[-1] = tot_adb_amt.iloc[-1]/active_acct_cnt.iloc[-1]
acct_adb_amt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,0.0,0.0,0.0,711.64000,796.510000,758.340000,749.890000,836.620000,701.510000,491.41000,...,1320.980000,1842.140000,2055.070000,2093.590000,2090.200000,1458.990000,1393.070000,1598.68000,2241.570000,1562.1300
202002,NaN,0.0,0.0,765.61000,813.880000,867.200000,930.080000,969.960000,1026.270000,634.61000,...,1196.730000,1358.460000,1419.070000,1470.120000,1554.210000,1457.100000,1548.300000,1535.24000,1584.540000,1556.3900
202003,NaN,NaN,0.0,474.93000,748.600000,810.640000,864.580000,920.870000,972.340000,589.63000,...,1134.600000,1257.450000,1374.120000,1361.970000,1381.950000,1396.000000,1477.450000,1540.83000,1610.780000,1605.0700
202004,NaN,NaN,NaN,0.00000,644.990000,828.260000,910.670000,970.070000,1044.550000,747.38000,...,1320.800000,1482.860000,1564.900000,1557.480000,1588.700000,1563.490000,1652.720000,1698.48000,1738.430000,1695.4500
202005,NaN,NaN,NaN,NaN,0.000000,628.880000,813.390000,886.220000,999.020000,822.96000,...,1335.740000,1432.390000,1529.400000,1531.890000,1661.720000,1604.010000,1767.610000,1822.95000,1838.820000,1810.5400
202006,NaN,NaN,NaN,NaN,NaN,0.000000,491.990000,718.830000,838.980000,830.01000,...,1411.620000,1527.510000,1600.210000,1630.200000,1730.630000,1763.500000,1917.490000,2011.93000,1974.740000,1995.6100
202007,NaN,NaN,NaN,NaN,NaN,NaN,0.680000,447.650000,763.220000,866.89000,...,1446.920000,1586.380000,1640.490000,1620.190000,1581.040000,1713.290000,1729.660000,1731.50000,1732.510000,1760.4400
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,305.610000,480.29000,...,1007.260000,1158.180000,1208.730000,1216.340000,1247.460000,1232.580000,1446.310000,1556.32000,1522.740000,1502.0000
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,234.17000,...,916.970000,1065.480000,1146.260000,1144.320000,1190.040000,1231.300000,1326.310000,1405.02000,1452.160000,1453.8200


In [35]:
adb_cl_util_pct = pd.pivot_table(all_df_1,values='adb_cl_util_pct',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
adb_cl_util_pct.iloc[-1] = tot_adb_amt.iloc[-1]/active_credit_limit.iloc[-1]
adb_cl_util_pct

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,0.0,0.0,0.0,0.167400,0.195200,0.149500,0.145500,0.175400,0.123100,0.086500,...,0.157200,0.263400,0.242000,0.243200,0.2207,0.159400,0.16060,0.166600,0.225700,0.189800
202002,NaN,0.0,0.0,0.403500,0.429800,0.464600,0.473100,0.499300,0.528600,0.328100,...,0.380000,0.420700,0.437000,0.447100,0.4721,0.424600,0.44600,0.439900,0.448900,0.434000
202003,NaN,NaN,0.0,0.291300,0.460900,0.501400,0.520400,0.558100,0.588500,0.357000,...,0.415100,0.448100,0.487100,0.481100,0.4863,0.462900,0.49130,0.495300,0.518000,0.515800
202004,NaN,NaN,NaN,0.000000,0.334600,0.434000,0.474800,0.502700,0.537800,0.387100,...,0.439000,0.482100,0.504800,0.503000,0.5076,0.475400,0.49950,0.512300,0.516100,0.503900
202005,NaN,NaN,NaN,NaN,0.000000,0.297300,0.382900,0.418200,0.461800,0.383800,...,0.401300,0.426100,0.447100,0.445200,0.4816,0.435100,0.47270,0.484300,0.487800,0.468700
202006,NaN,NaN,NaN,NaN,NaN,0.000000,0.222000,0.325900,0.364200,0.361600,...,0.397100,0.423300,0.440700,0.449800,0.4721,0.454400,0.49180,0.505100,0.497200,0.497000
202007,NaN,NaN,NaN,NaN,NaN,NaN,0.000200,0.189900,0.326200,0.367900,...,0.417900,0.452800,0.468400,0.455000,0.4385,0.448200,0.45820,0.456700,0.452700,0.458600
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.172800,0.271700,...,0.346800,0.398100,0.411600,0.413800,0.4189,0.378600,0.43850,0.468200,0.456300,0.439500
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.152300,...,0.344500,0.390700,0.423000,0.418200,0.4342,0.402100,0.43400,0.451200,0.462600,0.458400


In [36]:
tot_prncp_amt = pd.pivot_table(all_df_1,values='tot_prncp_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
tot_prncp_amt

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,133669.28,185238.45,170865.14,155728.06,114031.95,130713.25,144190.57,140187.79,152964.65,154704.69,...,2.072911e+05,2.078732e+05,1.822951e+05,2.158631e+05,1.873243e+05,1.525112e+05,1.665947e+05,1.927033e+05,2.063622e+05,1.851341e+05
202002,NaN,1531684.14,2036064.38,1874020.19,1846986.64,1934297.48,1923826.65,2018305.18,2109831.15,2069730.21,...,1.750667e+06,1.896491e+06,1.817106e+06,1.892413e+06,1.866107e+06,1.914054e+06,1.983783e+06,1.929526e+06,1.865571e+06,1.885069e+06
202003,NaN,NaN,4733916.13,6769075.88,6334532.66,6740476.41,6842884.19,7018288.08,7305315.98,7043441.57,...,5.751446e+06,6.085266e+06,5.990980e+06,6.016591e+06,5.877112e+06,6.113741e+06,6.219541e+06,6.335621e+06,6.353412e+06,6.113690e+06
202004,NaN,NaN,NaN,5089824.43,5908059.52,6244099.78,6433300.10,6546937.95,6888642.30,6632182.92,...,5.694741e+06,5.951167e+06,5.856856e+06,5.803553e+06,5.660885e+06,5.938445e+06,6.002490e+06,5.934768e+06,5.974867e+06,5.762326e+06
202005,NaN,NaN,NaN,NaN,2480035.05,2854735.11,2909617.51,3003485.30,2416633.77,3047801.30,...,2.673724e+06,2.805416e+06,2.826911e+06,2.865397e+06,2.745280e+06,2.936924e+06,3.010379e+06,2.942832e+06,3.054406e+06,2.969354e+06
202006,NaN,NaN,NaN,NaN,NaN,2577710.57,3182305.02,3204548.10,3658492.83,3515645.84,...,3.254801e+06,3.272796e+06,3.265021e+06,3.271086e+06,3.152140e+06,3.336223e+06,3.391225e+06,3.375944e+06,3.418708e+06,3.340805e+06
202007,NaN,NaN,NaN,NaN,NaN,NaN,1519291.59,2130421.73,2304467.38,2152760.29,...,2.013555e+06,2.025295e+06,2.009822e+06,1.998514e+06,2.006480e+06,2.106813e+06,2.080110e+06,2.010428e+06,2.021871e+06,1.942908e+06
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2292491.94,2970086.70,2854060.10,...,2.876715e+06,2.971493e+06,2.940722e+06,2.947063e+06,2.865769e+06,3.090772e+06,3.187502e+06,3.206671e+06,3.130095e+06,3.011965e+06
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1941490.31,2226158.00,...,2.583850e+06,2.708743e+06,2.649101e+06,2.692875e+06,2.574585e+06,2.889765e+06,2.892476e+06,2.960415e+06,2.935610e+06,2.820324e+06


In [37]:
all_adb = pd.pivot_table(all_df_1,values='all_adb_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)

revolving_pct = pd.pivot_table(all_df_1,values='revolving_rate',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
revolving_pct.iloc[-1] = tot_adb_amt.iloc[-1]/all_adb.iloc[-1]
revolving_pct

stmt_mth,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
vintage,,,,,,,,,,,,,,,,,,,,,
202001,0.0,0.0,0.0,0.379100,0.50890,0.525600,0.456100,0.484700,0.404200,0.271800,...,0.418500,0.522100,0.606700,0.583600,0.613600,0.5560,0.507600,0.531600,0.652800,0.48630
202002,NaN,0.0,0.0,0.632000,0.69000,0.704500,0.717700,0.708300,0.726900,0.427400,...,0.592300,0.634600,0.654500,0.656500,0.676900,0.6303,0.639100,0.616200,0.645600,0.62380
202003,NaN,NaN,0.0,0.451000,0.69060,0.719000,0.731500,0.731100,0.747600,0.436200,...,0.634200,0.654700,0.691900,0.683500,0.681400,0.6678,0.685100,0.688800,0.700300,0.69670
202004,NaN,NaN,NaN,0.000000,0.54370,0.655900,0.688400,0.701900,0.728500,0.501200,...,0.655900,0.689100,0.711100,0.703600,0.715100,0.6775,0.688700,0.693200,0.700500,0.67600
202005,NaN,NaN,NaN,NaN,0.00000,0.494700,0.598400,0.622900,0.668200,0.535400,...,0.627600,0.637700,0.657500,0.656400,0.699000,0.6591,0.682000,0.686700,0.672700,0.65200
202006,NaN,NaN,NaN,NaN,NaN,0.000000,0.390900,0.527900,0.561900,0.522200,...,0.625700,0.638900,0.654200,0.657600,0.686900,0.6710,0.695800,0.717500,0.697000,0.69990
202007,NaN,NaN,NaN,NaN,NaN,NaN,0.000600,0.324700,0.509900,0.553100,...,0.631000,0.654700,0.663800,0.642800,0.620400,0.6331,0.626500,0.620000,0.623500,0.62770
202008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.288900,0.423400,...,0.560500,0.601400,0.612800,0.608500,0.613200,0.5873,0.631200,0.656200,0.637700,0.61580
202009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.248700,...,0.544300,0.577400,0.613500,0.603300,0.614200,0.5954,0.609200,0.624900,0.630900,0.61890
